### Open Flights Data Wrangling

To practice, you are going to wrangle data from OpenFlights.  You can read about it here: 

http://openflights.org/data.html

This includes five files: 

1. A file with one record for each airport, 
2. A file with one record for each airline, 
3. A file with one record for each each route, 
4. A file with one record for each type of airplane, and
5. A file with one record for each country.  

Each file has a relationship to the others, with some fields in common.  To make best use of the data, we will need to combine all three files.  This is the core challenge of relational databases.  This lesson looks at how to do that, specifically using the merge() function in python.  

Credits:

This lesson draws from the Data Analysis and Visualization in Python for Ecologists course by Data Carpentry:

https://datacarpentry.org/python-ecology-lesson/05-merging-data/index.html

and from the pandas documentation:

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html


### Learning Objectives: 

1. Understand the meaning of relational data and be able to identify common keys. 

2. Understand the different types of relational joins: concatenate, inner join, outer join, left join, right join. 

3. Use the pandas syntax pd.concat() and pd.merge() to perform those relational joins. 

4. Start to build an awareness of open data resources, and of documentation/tutorials for ongoing learning. 

In [1]:
import pandas as pd
import numpy as np

In [2]:
# These files use \N as a missing value indicator.  When reading the CSVs, we will tell
# it to use that value as missing or NA.  The double backslash is required because
# otherwise it will interpret \N as a carriage return. 

# Read in the airports data.
airports = pd.read_csv("data/airports.dat", header=None, na_values='\\N')
airports.columns = ["ID", "NAME", "CITY", "COUNTRY", "IATA", "ICAO", "LATITUDE", "LONGITUDE", "ALTITUDE", "TIMEZONE", "DST", "TZ", "TYPE", "SOURCE"]

# Read in the airlines data.
airlines = pd.read_csv("data/airlines.dat", header=None, na_values='\\N')
airlines.columns = ["ID", "NAME", "ALIAS", "IATA", "ICAO", "CALLSIGN", "COUNTRY", "ACTIVE"]

# Read in the routes data.
routes = pd.read_csv("data/routes.dat", header=None, na_values='\\N')
routes.columns = ["AIRLINE", "AIRLINE_ID", "SOURCE", "SOURCE_ID", "DEST", "DEST_ID", "CODESHARE", "STOPS", "EQUIPMENT"]

# Read in the planes data.
planes = pd.read_csv("data/planes.dat", header=None, na_values='\\N')
planes.columns = ["NAME", "IATA", "ICAO"]

# Read in the countries data.
countries = pd.read_csv("data/countries.dat", header=None, na_values='\\N')
countries.columns = ["NAME", "ISO", "DAFIF"]

### Start by seeing what's in the data.  

What columns are there?  What data types are the columns?  

Remember, 'object' means it is a string, while the numerical values can be floats or ints.  Sometimes you will have problems if it reads numeric data in as strings.  If that happens, you can use the function .astype() to convert it.  Look it up in the pandas API to get more details

Start with the airports table

In [3]:
# printing the first few rows gives us a good sense of what is in there
airports.head()

,ID,NAME,CITY,COUNTRY,IATA,ICAO,LATITUDE,LONGITUDE,ALTITUDE,TIMEZONE,DST,TZ,TYPE,SOURCE
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10.0,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10.0,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10.0,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10.0,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10.0,U,Pacific/Port_Moresby,airport,OurAirports


In [4]:
# we can also look a the data types
# alternatively, we could summarize the data as we did in the previous lesson
airports.dtypes

ID             int64
NAME          object
CITY          object
COUNTRY       object
IATA          object
ICAO          object
LATITUDE     float64
LONGITUDE    float64
ALTITUDE       int64
TIMEZONE     float64
DST           object
TZ            object
TYPE          object
SOURCE        object
dtype: object

##### Your turn:  

Spend a few minutes examining the remaining tables.  See if you can identify which fields are common across multiple tables.  These are the "keys".  Together, we will make sure we have identified the right keys. 

In [5]:
airlines.head()

,ID,NAME,ALIAS,IATA,ICAO,CALLSIGN,COUNTRY,ACTIVE
0,-1,Unknown,NaN,-,NaN,NaN,NaN,Y
1,1,Private flight,NaN,-,NaN,NaN,NaN,Y
2,2,135 Airways,NaN,NaN,GNL,GENERAL,United States,N
3,3,1Time Airline,NaN,1T,RNX,NEXTIME,South Africa,Y
4,4,2 Sqn No 1 Elementary Flying Training School,NaN,NaN,WYT,NaN,United Kingdom,N


In [6]:
routes.head()

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
0,2B,410.0,AER,2965.0,KZN,2990.0,NaN,0,CR2
1,2B,410.0,ASF,2966.0,KZN,2990.0,NaN,0,CR2
2,2B,410.0,ASF,2966.0,MRV,2962.0,NaN,0,CR2
3,2B,410.0,CEK,2968.0,KZN,2990.0,NaN,0,CR2
4,2B,410.0,CEK,2968.0,OVB,4078.0,NaN,0,CR2


In [7]:
planes.head()

,NAME,IATA,ICAO
0,Aerospatiale (Nord) 262,ND2,N262
1,Aerospatiale (Sud Aviation) Se.210 Caravelle,CRV,S210
2,Aerospatiale SN.601 Corvette,NDC,S601
3,Aerospatiale/Alenia ATR 42-300,AT4,AT43
4,Aerospatiale/Alenia ATR 42-500,AT5,AT45


In [8]:
countries.head()

,NAME,ISO,DAFIF
0,"Bonaire, Saint Eustatius and Saba",BQ,NaN
1,Aruba,AW,AA
2,Antigua and Barbuda,AG,AC
3,United Arab Emirates,AE,AE
4,Afghanistan,AF,AF


### Selecting data

We might want to work with a subset of data.  If so, we can select data just like we did in the previous lesson. 

In [9]:
# we can select all airports in one of the least visited countries on earth
airports[airports['COUNTRY']=="Tuvalu"]

,ID,NAME,CITY,COUNTRY,IATA,ICAO,LATITUDE,LONGITUDE,ALTITUDE,TIMEZONE,DST,TZ,TYPE,SOURCE
3835,4077,Funafuti International Airport,Funafuti,Tuvalu,FUN,NGFU,-8.525,179.195999,9,12.0,U,Pacific/Funafuti,airport,OurAirports


In [10]:
# we can select all routes to that airport
routes[routes['DEST']=='FUN']

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
24819,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0,AT5


In [11]:
# and routes from that airport
routes[routes['SOURCE']=='FUN']

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
24784,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0,AT5


In [12]:
# where are these flights going to/from? 


#####  Your turn: 

Select just the routes that go to or from Lexington Bluegrass Airport, and store them in their own dataframe.  

The airport code is LEX.  You should have a much smaller dataframe.  How many inbound routes and how many outbound routes are there? 

In [13]:
lex_from = routes[routes['SOURCE'] == 'LEX'] 
lex_from

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
3588,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ
5763,AA,24.0,LEX,4017.0,CLT,3876.0,Y,0,CR7 CRJ
5764,AA,24.0,LEX,4017.0,DFW,3670.0,Y,0,ERD ER4
5765,AA,24.0,LEX,4017.0,ORD,3830.0,Y,0,ERD ER4
9641,AF,137.0,LEX,4017.0,ATL,3682.0,Y,0,CRJ CR9
21095,DL,2009.0,LEX,4017.0,ATL,3682.0,NaN,0,M88 717
21096,DL,2009.0,LEX,4017.0,DCA,3520.0,Y,0,CRJ
21097,DL,2009.0,LEX,4017.0,DTW,3645.0,Y,0,CR7 CRJ CR9
21098,DL,2009.0,LEX,4017.0,LGA,3697.0,NaN,0,ERJ
21099,DL,2009.0,LEX,4017.0,MSP,3858.0,Y,0,CRJ


In [14]:
#outbound routes; from lexington airport
len(lex_from)

20

In [15]:
lex_to = routes[routes['DEST'] == 'LEX']
lex_to

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
3569,9E,3976.0,ATL,3682.0,LEX,4017.0,NaN,0,CRJ
4953,AA,24.0,CLT,3876.0,LEX,4017.0,NaN,0,CR7
5247,AA,24.0,DFW,3670.0,LEX,4017.0,Y,0,ERD ER4
6283,AA,24.0,ORD,3830.0,LEX,4017.0,Y,0,ERD ER4
9097,AF,137.0,ATL,3682.0,LEX,4017.0,Y,0,CR9 M88 CRJ 717
20164,DL,2009.0,ATL,3682.0,LEX,4017.0,NaN,0,M88 717
20534,DL,2009.0,DCA,3520.0,LEX,4017.0,Y,0,CRJ
20638,DL,2009.0,DTW,3645.0,LEX,4017.0,NaN,0,717
21131,DL,2009.0,LGA,3697.0,LEX,4017.0,Y,0,ERJ
21402,DL,2009.0,MSP,3858.0,LEX,4017.0,Y,0,CRJ


In [16]:
#inbound routes; coming to lexington airport
len(lex_to)

20

### Concatentating data

To concatenate is to link things together in a series or chain.  It is similar to, but more powerful than appending data, because it can work on either rows or columns, is faster, and will perform set logic (union or intersection).  

In [17]:
# let's go back and contatenate the routes both to and from Tuvalu
to_tuvalu = routes[routes['DEST']=='FUN']
from_tuvalu = routes[routes['SOURCE']=='FUN']


In [18]:
# the basic function, given a list of dataframes to concatenate
# note that we call concatenate as a "static" method, rather than a method 
# associated with a specific object. 

pd.concat([to_tuvalu, from_tuvalu])

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
24819,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0,AT5
24784,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0,AT5


In [19]:
# if we want to use the dataframe later, we need to assign it to a variable name

tuvalu_routes = pd.concat([to_tuvalu, from_tuvalu])

Note to see more about this method, we can examine the pandas docs:

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [20]:
# note that one of the arguments is axis.  
# axis=0 indicates that the rows should be concatenated, 
# while axis=1 indicates to concatenate the columns 

pd.concat([to_tuvalu, from_tuvalu], axis=1)

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
24784,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0.0,AT5
24819,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0.0,AT5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### Your turn:

Can you use the concat() function to create one dataframe with flights both into and out of LEX?

In [21]:
lex_routes = pd.concat([lex_to, lex_from])
lex_routes

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
3569,9E,3976.0,ATL,3682.0,LEX,4017.0,NaN,0,CRJ
4953,AA,24.0,CLT,3876.0,LEX,4017.0,NaN,0,CR7
5247,AA,24.0,DFW,3670.0,LEX,4017.0,Y,0,ERD ER4
6283,AA,24.0,ORD,3830.0,LEX,4017.0,Y,0,ERD ER4
9097,AF,137.0,ATL,3682.0,LEX,4017.0,Y,0,CR9 M88 CRJ 717
20164,DL,2009.0,ATL,3682.0,LEX,4017.0,NaN,0,M88 717
20534,DL,2009.0,DCA,3520.0,LEX,4017.0,Y,0,CRJ
20638,DL,2009.0,DTW,3645.0,LEX,4017.0,NaN,0,717
21131,DL,2009.0,LGA,3697.0,LEX,4017.0,Y,0,ERJ
21402,DL,2009.0,MSP,3858.0,LEX,4017.0,Y,0,CRJ


### Merging data

Sometimes we want to merge, or join, the data instead of concatenating it.  For example, maybe we hate flying on small planes, so want to know what kind of aircraft we would need to fly on to visit Tuvalu.  This data comes from a different table. 

In [22]:
# we see that the Tuvalu routes operate with AT5 equipment
tuvalu_routes

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
24819,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0,AT5
24784,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0,AT5


In [23]:
# but what is an AT5? Let's look in the planes table. 
planes.head()

,NAME,IATA,ICAO
0,Aerospatiale (Nord) 262,ND2,N262
1,Aerospatiale (Sud Aviation) Se.210 Caravelle,CRV,S210
2,Aerospatiale SN.601 Corvette,NDC,S601
3,Aerospatiale/Alenia ATR 42-300,AT4,AT43
4,Aerospatiale/Alenia ATR 42-500,AT5,AT45


In [24]:
# in this case, we are lucky, and it is in the first few rows.  But to do this systemactically, we
# would want to join the tables.  To do this, we use the merge() function in pandas.  

pd.merge(tuvalu_routes, planes, left_on='EQUIPMENT', right_on='IATA')

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,NAME,IATA,ICAO
0,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45
1,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45


Let's stop and look at what is going on in the above command. 

It is important to understand the type of merge, as specified by the how option.  Here is what the pandas docs say:


![alt text](img/how.png)


![alt text](img/how2.png)

So it looks like the above merge defaulted to an inner join


![alt text](img/inner.png)

In [25]:
# we can check this by explicitly setting the option and confirming that we get the same result  
# next, let's check what happens if we choose the other options

pd.merge(tuvalu_routes, planes, left_on='EQUIPMENT', right_on='IATA', how='inner')

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,NAME,IATA,ICAO
0,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45
1,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45


In [26]:
# outer join

pd.merge(tuvalu_routes, planes, left_on='EQUIPMENT', right_on='IATA', how='outer')

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,NAME,IATA,ICAO
0,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0.0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45
1,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0.0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale (Nord) 262,ND2,N262
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale (Sud Aviation) Se.210 Caravelle,CRV,S210
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale SN.601 Corvette,NDC,S601
...,...,...,...,...,...,...,...,...,...,...,...,...
242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupolev Tu-134,TU3,T134
243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupolev Tu-154,TU5,T154
244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupolev Tu-204,T20,T204
245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yakovlev Yak-40,YK4,YK40


In [27]:
# left join

pd.merge(tuvalu_routes, planes, left_on='EQUIPMENT', right_on='IATA', how='left')

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,NAME,IATA,ICAO
0,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45
1,FJ,879.0,FUN,4077.0,SUV,1961.0,NaN,0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45


In [28]:
# right join

pd.merge(tuvalu_routes, planes, left_on='EQUIPMENT', right_on='IATA', how='right')

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,NAME,IATA,ICAO
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale (Nord) 262,ND2,N262
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale (Sud Aviation) Se.210 Caravelle,CRV,S210
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale SN.601 Corvette,NDC,S601
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerospatiale/Alenia ATR 42-300,AT4,AT43
4,FJ,879.0,SUV,1961.0,FUN,4077.0,NaN,0.0,AT5,Aerospatiale/Alenia ATR 42-500,AT5,AT45
...,...,...,...,...,...,...,...,...,...,...,...,...
242,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupolev Tu-144,NaN,T144
243,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupolev Tu-154,TU5,T154
244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tupolev Tu-204,T20,T204
245,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yakovlev Yak-40,YK4,YK40


I find that I most often want a left join, as described here: 

![alt text](img/left.png)

It may be worth examining some of the other options used in merge().  We can refer to the pandas documentation to learn more. 

https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html#database-style-dataframe-or-named-series-joining-merging

#### Your turn

Now let's look at which airlines operate in and out of Lexington.  How many routes does each airline have?  

First, let's spend some time planning together.  Then individually take some time to implement the code we planned. 

In [29]:
lex_routes.AIRLINE.unique()

array(['9E', 'AA', 'AF', 'DL', 'G4', 'KL', 'UA', 'US'], dtype=object)

In [30]:
lex_routes['AIRLINE'].value_counts()

DL    10
G4     8
AA     6
US     6
UA     4
9E     2
AF     2
KL     2
Name: AIRLINE, dtype: int64

#### It looks like there are some international airlines with Lexington routes.  

To look at how many routes they have, create a new column in your dataframe called 'International', which is set to True for an overseas airline and False for a domestic airline.  Calculate the percent of routes with an overseas airline.  

You should have the tools you need to answer this question using what we learned in last week's lesson.  Again, let's start by planning together, then implement the code on your own.  

In [31]:
#First method of solving it 
airlines[airlines['COUNTRY'] == 'United States']

,ID,NAME,ALIAS,IATA,ICAO,CALLSIGN,COUNTRY,ACTIVE
2,2,135 Airways,NaN,NaN,GNL,GENERAL,United States,N
9,9,3D Aviation,NaN,NaN,SEC,SECUREX,United States,N
10,10,40-Mile Air,NaN,Q5,MLA,MILE-AIR,United States,Y
22,22,Aloha Airlines,NaN,AQ,AAH,ALOHA,United States,Y
23,23,Alaska Island Air,NaN,NaN,AAK,ALASKA ISLAND,United States,N
...,...,...,...,...,...,...,...,...
6141,20978,All America US,All America US,AG,SSA,NaN,United States,Y
6143,20991,Choice Airways,NaN,NaN,CSX,NaN,United States,Y
6146,20996,Dynamic Airways,NaN,NaN,DYA,NaN,United States,Y
6152,21124,ViaAir,Charter Air Transport,NaN,SRY,Stingray,United States,Y


In [32]:
domestic = airlines[airlines['COUNTRY'] == 'United States'].IATA.unique()
domestic

array([nan, 'Q5', 'AQ', 'AA', 'G4', 'GB', '4G', '7A', 'KO', '2Q', '5A',
       'M6', 'HP', 'ZW', 'ZV', '6V', 'JW', 'EV', '8C', 'MQ', '5F', 'ZA',
       'ER', 'QH', '5Y', 'MC', 'FL', '8Q', 'B4', 'CH', '8E', 'BN', 'GQ',
       'E9', 'R9', '9K', 'PT', 'GG', 'WE', 'OP', 'MG', 'RP', 'C8', 'DQ',
       '9L', 'OH', 'C5', 'CP', 'CO', 'CS', 'DL', 'DH', 'W9', 'EM', 'E0',
       'EZ', 'OW', 'EO', 'XE', 'FX', 'PA', 'RF', 'HK', 'FP', 'F9', '2F',
       '1G', 'GR', 'G7', 'GS', 'ZK', 'H6', 'HB', 'HA', 'QX', 'II', '1U',
       'I9', 'IS', '2S', 'QJ', 'B6', 'K4', 'M5', 'KP', 'JF', 'L2', 'MY',
       'YV', 'XJ', 'GL', 'JI', 'YX', 'DV', 'N7', '1N', '1I', 'EJ', 'NC',
       'NW', 'OY', 'OZ', 'Y5', 'PS', 'LW', 'KS', 'PI', '9E', 'PO', 'BK',
       'FE', '1D', '3C', 'QQ', 'RW', 'E2', 'RD', 'BB', 'SY', 'N5', 'WN',
       'AL', 'SO', '2G', '1S', '1K', 'NK', 'SX', 'OO', '8D', 'S5', 'TI',
       'FF', 'T9', 'TW', 'AX', '1L', 'U5', 'UA', 'U2', 'U7', '5X', 'US',
       'UH', 'VX', 'WA', 'CN', 'K5', 'WO', '1P', '8V

In [33]:
type(lex_routes['AIRLINE'][3569])

str

In [34]:
def is_international(a):
    '''
    domestic is a list of two digit IATA codes of US carriers
    a represents a two digit IATA code 
    This method will return false for domestic flights and true for international flights
    '''
    if a in domestic:
        return False
    else:
        return True

In [35]:
lex_routes['INTL'] = lex_routes['AIRLINE'].apply(is_international)

In [36]:
lex_routes

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,INTL
3569,9E,3976.0,ATL,3682.0,LEX,4017.0,NaN,0,CRJ,False
4953,AA,24.0,CLT,3876.0,LEX,4017.0,NaN,0,CR7,False
5247,AA,24.0,DFW,3670.0,LEX,4017.0,Y,0,ERD ER4,False
6283,AA,24.0,ORD,3830.0,LEX,4017.0,Y,0,ERD ER4,False
9097,AF,137.0,ATL,3682.0,LEX,4017.0,Y,0,CR9 M88 CRJ 717,True
20164,DL,2009.0,ATL,3682.0,LEX,4017.0,NaN,0,M88 717,False
20534,DL,2009.0,DCA,3520.0,LEX,4017.0,Y,0,CRJ,False
20638,DL,2009.0,DTW,3645.0,LEX,4017.0,NaN,0,717,False
21131,DL,2009.0,LGA,3697.0,LEX,4017.0,Y,0,ERJ,False
21402,DL,2009.0,MSP,3858.0,LEX,4017.0,Y,0,CRJ,False


In [37]:
#Second method of solving it
lex_routes.merge(airlines, how='left', left_on='AIRLINE_ID', right_on='ID')

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,INTL,ID,NAME,ALIAS,IATA,ICAO,CALLSIGN,COUNTRY,ACTIVE
0,9E,3976.0,ATL,3682.0,LEX,4017.0,NaN,0,CRJ,False,3976,Pinnacle Airlines,NaN,9E,FLG,FLAGSHIP,United States,Y
1,AA,24.0,CLT,3876.0,LEX,4017.0,NaN,0,CR7,False,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y
2,AA,24.0,DFW,3670.0,LEX,4017.0,Y,0,ERD ER4,False,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y
3,AA,24.0,ORD,3830.0,LEX,4017.0,Y,0,ERD ER4,False,24,American Airlines,NaN,AA,AAL,AMERICAN,United States,Y
4,AF,137.0,ATL,3682.0,LEX,4017.0,Y,0,CR9 M88 CRJ 717,True,137,Air France,NaN,AF,AFR,AIRFRANS,France,Y
5,DL,2009.0,ATL,3682.0,LEX,4017.0,NaN,0,M88 717,False,2009,Delta Air Lines,NaN,DL,DAL,DELTA,United States,Y
6,DL,2009.0,DCA,3520.0,LEX,4017.0,Y,0,CRJ,False,2009,Delta Air Lines,NaN,DL,DAL,DELTA,United States,Y
7,DL,2009.0,DTW,3645.0,LEX,4017.0,NaN,0,717,False,2009,Delta Air Lines,NaN,DL,DAL,DELTA,United States,Y
8,DL,2009.0,LGA,3697.0,LEX,4017.0,Y,0,ERJ,False,2009,Delta Air Lines,NaN,DL,DAL,DELTA,United States,Y
9,DL,2009.0,MSP,3858.0,LEX,4017.0,Y,0,CRJ,False,2009,Delta Air Lines,NaN,DL,DAL,DELTA,United States,Y


#### Codeshares

It looks like a bunch of these routes are codeshares.  That means they are marketed by this airline, but operated by a different airline.  See the note in the data documentation on openflights.org/data.  The implication of this is that there are duplicates.

Can you figure out which ones are duplicates?  Can you then create a dataframe with only the unique routes?  How many unique inbound and outbound routes are there? 

Remember, someone has to operate the flight, so if all the routes to/from a particular airport are listed as codeshares, then something is funny...

It is also possible that more than one airline actually operates a route between the same two airports. (Having this sort of competition generally means that you will get better fares as a traveler.)  It may not be obvious what is actually in the data set, so dig or do external research as needed.  

It is this sort of messiness that will consume much of your effort in research and data science.  It is very important to "become one with your data". 

Figuring this out will be the first part of your homework.  

### Review Questions: 

1. What is a key in relational data? 

2. Explain the difference between a left join and an outer join.  

3. Explain the difference between pd.concat() and pd.merge()

4. Identify two possible places to look for help if you get stuck on the homework. 

In [42]:
# A key in relational data is something that has a corresponding value in a list or dictionary

In [43]:
# Left join uses the keys from the left frame only while outer join uses union of keys from both frames and 
# includes all the data that are null values as well

In [44]:
# pd.concat basically links all data together into one dataframe and combining them under similar columns
# while pd.merge brings two different data tables together into one table 
# and then data are all under their respective columns

In [45]:
# I can google search any help needed and also I can make use of the documentation user guide packages

### Homework 1 - Code shares

Dig into the data as described above, and do your best to identify which routes are code shares, and how many unique routes operate into/out of Lexington.

In [38]:
codeshares = lex_routes[lex_routes['CODESHARE'] == 'Y']
codeshares

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,INTL
5247,AA,24.0,DFW,3670.0,LEX,4017.0,Y,0,ERD ER4,False
6283,AA,24.0,ORD,3830.0,LEX,4017.0,Y,0,ERD ER4,False
9097,AF,137.0,ATL,3682.0,LEX,4017.0,Y,0,CR9 M88 CRJ 717,True
20534,DL,2009.0,DCA,3520.0,LEX,4017.0,Y,0,CRJ,False
21131,DL,2009.0,LGA,3697.0,LEX,4017.0,Y,0,ERJ,False
21402,DL,2009.0,MSP,3858.0,LEX,4017.0,Y,0,CRJ,False
36396,KL,3090.0,ATL,3682.0,LEX,4017.0,Y,0,CR9 M88 CRJ,True
56775,UA,5209.0,IAH,3550.0,LEX,4017.0,Y,0,ERJ,False
57322,UA,5209.0,ORD,3830.0,LEX,4017.0,Y,0,ERJ CRJ,False
59924,US,5265.0,ORD,3830.0,LEX,4017.0,Y,0,ERD ER4,False


In [39]:
notcodeshares = lex_routes[lex_routes['CODESHARE'] != 'Y']
notcodeshares

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT,INTL
3569,9E,3976.0,ATL,3682.0,LEX,4017.0,NaN,0,CRJ,False
4953,AA,24.0,CLT,3876.0,LEX,4017.0,NaN,0,CR7,False
20164,DL,2009.0,ATL,3682.0,LEX,4017.0,NaN,0,M88 717,False
20638,DL,2009.0,DTW,3645.0,LEX,4017.0,NaN,0,717,False
28951,G4,35.0,FLL,3533.0,LEX,4017.0,NaN,0,M80,False
29090,G4,35.0,PGD,7056.0,LEX,4017.0,NaN,0,M80,False
29120,G4,35.0,PIE,3617.0,LEX,4017.0,NaN,0,320,False
29187,G4,35.0,SFB,4167.0,LEX,4017.0,NaN,0,M80 320,False
58812,US,5265.0,CLT,3876.0,LEX,4017.0,NaN,0,CR7,False
59096,US,5265.0,DFW,3670.0,LEX,4017.0,NaN,0,ERD,False


In [40]:
#unique routes that operate in/out of lexington
len(notcodeshares)

19

### Homework 2 - The Kevin Bacon Game

Of course, it is possible to fly beyond those few cities from Lexington--you just have to transfer to get there.  Your task is to:

1. Identify which airports you can get to from Lexington with only one transfer.  
2. Identify which airports you can get to from Lexington with two transfers.

You have the tools to do this using the merge functionality presented today, but you will need to be smart about what you are merging.  

Next class, we will use these data to make interactive maps of these flight routes, similar to what you see in the back of the airline magazines.  


#### Bonus: 

This is the air travel version of the Kevin Bacon game (https://oracleofbacon.org/). What is the number N, such that you can reach every airport in the world with N or fewer transfers?

#### Extra Bonus: 

Use this very important piece of knowledge to impress your friends at parties!

In [41]:
lex_from

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
3588,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ
5763,AA,24.0,LEX,4017.0,CLT,3876.0,Y,0,CR7 CRJ
5764,AA,24.0,LEX,4017.0,DFW,3670.0,Y,0,ERD ER4
5765,AA,24.0,LEX,4017.0,ORD,3830.0,Y,0,ERD ER4
9641,AF,137.0,LEX,4017.0,ATL,3682.0,Y,0,CRJ CR9
21095,DL,2009.0,LEX,4017.0,ATL,3682.0,NaN,0,M88 717
21096,DL,2009.0,LEX,4017.0,DCA,3520.0,Y,0,CRJ
21097,DL,2009.0,LEX,4017.0,DTW,3645.0,Y,0,CR7 CRJ CR9
21098,DL,2009.0,LEX,4017.0,LGA,3697.0,NaN,0,ERJ
21099,DL,2009.0,LEX,4017.0,MSP,3858.0,Y,0,CRJ


In [42]:
#This will concatate all the destinations you can get to from these airports and this will be 
#our base for one transfer routes from lexington
from_atl = routes[routes['SOURCE'] == 'ATL']
from_lga = routes[routes['SOURCE'] == 'LGA']
from_fll = routes[routes['SOURCE'] == 'FLL']
from_pgd = routes[routes['SOURCE'] == 'PGD']
from_pie = routes[routes['SOURCE'] == 'PIE']
from_sfb = routes[routes['SOURCE'] == 'SFB']
from_clt = routes[routes['SOURCE'] == 'CLT']
from_dfw = routes[routes['SOURCE'] == 'DFW']
from_iah = routes[routes['SOURCE'] == 'IAH']
from_msp = routes[routes['SOURCE'] == 'MSP']
from_dtw = routes[routes['SOURCE'] == 'DTW']
from_dca = routes[routes['SOURCE'] == 'DCA']
from_ord = routes[routes['SOURCE'] == 'ORD']
outboundlexroutes = pd.concat([from_dfw, from_iah, from_msp, from_dtw, from_dca, from_ord, from_clt, from_atl,from_sfb, from_pie, from_pgd, from_fll, from_lga])
outboundlexroutes

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
1059,4M,3201.0,DFW,3670.0,EZE,3988.0,Y,0,777
5160,AA,24.0,DFW,3670.0,ABI,3718.0,Y,0,ERD CRJ ER4
5161,AA,24.0,DFW,3670.0,ABQ,4019.0,NaN,0,M80 M83
5162,AA,24.0,DFW,3670.0,ACT,3700.0,Y,0,ER4 ERD
5163,AA,24.0,DFW,3670.0,AEX,3852.0,Y,0,CRJ ER4 ERD
...,...,...,...,...,...,...,...,...,...
64611,WS,5416.0,LGA,3697.0,ATL,3682.0,Y,0,M88 757 320 738 739 73W
64612,WS,5416.0,LGA,3697.0,MSY,3861.0,Y,0,319 CR9
64613,WS,5416.0,LGA,3697.0,YHZ,73.0,Y,0,CR7
64614,WS,5416.0,LGA,3697.0,YUL,146.0,Y,0,ERJ CR7


In [43]:
#drops any duplicates
outboundlexroutes = outboundlexroutes.drop_duplicates(subset = ['DEST'])
outboundlexroutes

,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
1059,4M,3201.0,DFW,3670.0,EZE,3988.0,Y,0,777
5160,AA,24.0,DFW,3670.0,ABI,3718.0,Y,0,ERD CRJ ER4
5161,AA,24.0,DFW,3670.0,ABQ,4019.0,NaN,0,M80 M83
5162,AA,24.0,DFW,3670.0,ACT,3700.0,Y,0,ER4 ERD
5163,AA,24.0,DFW,3670.0,AEX,3852.0,Y,0,CRJ ER4 ERD
...,...,...,...,...,...,...,...,...,...
43408,NK,4687.0,FLL,3533.0,CTG,2714.0,NaN,0,320 319
43418,NK,4687.0,FLL,3533.0,LBE,5747.0,NaN,0,319
43436,NK,4687.0,FLL,3533.0,STI,1764.0,NaN,0,319
43439,NK,4687.0,FLL,3533.0,TLC,1850.0,NaN,0,319


In [46]:
#this table merges airports you can go to direct from lex with the outboundlexroutes which shows their destinations
#this table shows other aitports you go to from lexington using one transfer
onetransferlex= pd.merge(lex_from, outboundlexroutes, left_on='DEST', right_on='SOURCE', how='inner')
onetransferlex

,AIRLINE_x,AIRLINE_ID_x,SOURCE_x,SOURCE_ID_x,DEST_x,DEST_ID_x,CODESHARE_x,STOPS_x,EQUIPMENT_x,AIRLINE_y,AIRLINE_ID_y,SOURCE_y,SOURCE_ID_y,DEST_y,DEST_ID_y,CODESHARE_y,STOPS_y,EQUIPMENT_y
0,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,3M,20710.0,ATL,3682.0,LWB,6958.0,NaN,0,SF3
1,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,3M,20710.0,ATL,3682.0,MCN,3754.0,NaN,0,SF3
2,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,3M,20710.0,ATL,3682.0,MEI,4335.0,NaN,0,SF3
3,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,3M,20710.0,ATL,3682.0,MSL,5756.0,NaN,0,SF3
4,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,3M,20710.0,ATL,3682.0,PIB,5759.0,NaN,0,SF3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,UA,5209.0,LEX,4017.0,IAH,3550.0,Y,0,ERJ,UA,5209.0,IAH,3550.0,UIO,2688.0,NaN,0,73G 738
733,UA,5209.0,LEX,4017.0,IAH,3550.0,Y,0,ERJ,UA,5209.0,IAH,3550.0,VER,1854.0,Y,0,ERJ
734,UA,5209.0,LEX,4017.0,IAH,3550.0,Y,0,ERJ,UA,5209.0,IAH,3550.0,VSA,1853.0,Y,0,CR7
735,UA,5209.0,LEX,4017.0,IAH,3550.0,Y,0,ERJ,UA,5209.0,IAH,3550.0,ZIH,1856.0,Y,0,ERJ


In [47]:
#This will be the list of airports used for two transfers
twotransfersources = onetransferlex['DEST_y'].unique()
print(twotransfersources)

['LWB' 'MCN' 'MEI' 'MSL' 'PIB' 'TUP' 'ABY' 'BQK' 'CSG' 'DHN' 'ECP' 'EYW'
 'GTR' 'STR' 'VLD' 'BDA' 'BSB' 'GGT' 'JNB' 'KIN' 'PAP' 'SDQ' 'CUR' 'ANU'
 'AUA' 'DAB' 'EWN' 'FLO' 'FPO' 'GNV' 'HTS' 'LYH' 'MLB' 'PGV' 'PHF' 'SBY'
 'TRI' 'UVF' 'LUK' 'EZE' 'ABI' 'ABQ' 'ACT' 'AEX' 'AGU' 'AMA' 'ATL' 'AUS'
 'BDL' 'BHM' 'BJX' 'BMI' 'BNA' 'BNE' 'BOG' 'BOS' 'BPT' 'BRO' 'BTR' 'BWI'
 'BZE' 'CAE' 'CCS' 'CDG' 'CHA' 'CHS' 'CID' 'CLE' 'CLL' 'CLT' 'CMH' 'CMI'
 'COS' 'COU' 'CRP' 'CRW' 'CUN' 'CUU' 'CVG' 'CZM' 'DAY' 'DCA' 'DEN' 'DRO'
 'DSM' 'DTW' 'ELP' 'EVV' 'EWR' 'FAR' 'FAT' 'FLL' 'FRA' 'FSD' 'FSM' 'FWA'
 'GCK' 'GDL' 'GGG' 'GIG' 'GJT' 'GPT' 'GRI' 'GRK' 'GRR' 'GRU' 'GSO' 'GSP'
 'GUA' 'HNL' 'HOU' 'HSV' 'IAD' 'IAH' 'ICN' 'ICT' 'IND' 'JAN' 'JAX' 'JFK'
 'JLN' 'LAS' 'LAW' 'LAX' 'LBB' 'LCH' 'LEX' 'LFT' 'LGA' 'LHR' 'LIM' 'LIR'
 'LIT' 'LRD' 'MAD' 'MAF' 'MBJ' 'MCI' 'MCO' 'MEM' 'MEX' 'MFE' 'MGM' 'MHK'
 'MIA' 'MKE' 'MLI' 'MLM' 'MLU' 'MOB' 'MSN' 'MSP' 'MSY' 'MTY' 'MZT' 'NAS'
 'NRT' 'OGG' 'OKC' 'OMA' 'ONT' 'ORD' 'ORF' 'PBC' 'P

In [51]:
#Merging the destinations in onetransfer lex (which will be used as sources for two transfers), 
#with the sources in the routes database to show us where you can get to from lex with two transfers
twotransfer_routes = pd.merge(onetransferlex, routes, left_on='DEST_y', right_on='SOURCE', how='inner')
twotransfer_routes

,AIRLINE_x,AIRLINE_ID_x,SOURCE_x,SOURCE_ID_x,DEST_x,DEST_ID_x,CODESHARE_x,STOPS_x,EQUIPMENT_x,AIRLINE_y,...,EQUIPMENT_y,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
0,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,3M,...,SF3,3M,20710.0,LWB,6958.0,ATL,3682.0,NaN,0,SF3
1,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,3M,...,SF3,UA,5209.0,LWB,6958.0,IAD,3714.0,Y,0,SF3
2,AF,137.0,LEX,4017.0,ATL,3682.0,Y,0,CRJ CR9,3M,...,SF3,3M,20710.0,LWB,6958.0,ATL,3682.0,NaN,0,SF3
3,AF,137.0,LEX,4017.0,ATL,3682.0,Y,0,CRJ CR9,3M,...,SF3,UA,5209.0,LWB,6958.0,IAD,3714.0,Y,0,SF3
4,DL,2009.0,LEX,4017.0,ATL,3682.0,NaN,0,M88 717,3M,...,SF3,3M,20710.0,LWB,6958.0,ATL,3682.0,NaN,0,SF3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53026,UA,5209.0,LEX,4017.0,IAH,3550.0,Y,0,ERJ,UA,...,ERJ,AM,321.0,ZLO,1858.0,MEX,1824.0,Y,0,E75 ERJ
53027,UA,5209.0,LEX,4017.0,IAH,3550.0,Y,0,ERJ,UA,...,ERJ,AS,439.0,ZLO,1858.0,LAX,3484.0,NaN,0,73H
53028,UA,5209.0,LEX,4017.0,IAH,3550.0,Y,0,ERJ,UA,...,ERJ,UA,5209.0,ZLO,1858.0,IAH,3550.0,Y,0,ERJ
53029,UA,5209.0,LEX,4017.0,IAH,3550.0,Y,0,ERJ,UA,...,ERJ,VW,1290.0,ZLO,1858.0,MEX,1824.0,NaN,0,ATR


In [52]:
#drops any duplicates
twotransfer_routes = twotransfer_routes.drop_duplicates(subset = ['DEST'])
twotransfer_routes

,AIRLINE_x,AIRLINE_ID_x,SOURCE_x,SOURCE_ID_x,DEST_x,DEST_ID_x,CODESHARE_x,STOPS_x,EQUIPMENT_x,AIRLINE_y,...,EQUIPMENT_y,AIRLINE,AIRLINE_ID,SOURCE,SOURCE_ID,DEST,DEST_ID,CODESHARE,STOPS,EQUIPMENT
0,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,3M,...,SF3,3M,20710.0,LWB,6958.0,ATL,3682.0,NaN,0,SF3
1,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,3M,...,SF3,UA,5209.0,LWB,6958.0,IAD,3714.0,Y,0,SF3
9,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,3M,...,SF3,3M,20710.0,MCN,3754.0,MCO,3878.0,NaN,0,SF3
29,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,3M,...,SF3,3M,20710.0,TUP,5773.0,GLH,6130.0,NaN,0,SF3
91,9E,3976.0,LEX,4017.0,ATL,3682.0,NaN,0,CRJ,AF,...,717 M88,WN,4547.0,ECP,7669.0,BNA,3690.0,NaN,0,73W
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52967,UA,5209.0,LEX,4017.0,IAH,3550.0,Y,0,ERJ,UA,...,73G 738,EQ,4863.0,UIO,2688.0,LGQ,6750.0,NaN,0,E70 E90
52969,UA,5209.0,LEX,4017.0,IAH,3550.0,Y,0,ERJ,UA,...,73G 738,EQ,4863.0,UIO,2688.0,LOH,6046.0,NaN,0,E90
52970,UA,5209.0,LEX,4017.0,IAH,3550.0,Y,0,ERJ,UA,...,73G 738,EQ,4863.0,UIO,2688.0,MEC,2683.0,NaN,0,E90
52972,UA,5209.0,LEX,4017.0,IAH,3550.0,Y,0,ERJ,UA,...,73G 738,EQ,4863.0,UIO,2688.0,TUA,2697.0,NaN,0,E70


In [53]:
twotransfer_routes.columns

Index(['AIRLINE_x', 'AIRLINE_ID_x', 'SOURCE_x', 'SOURCE_ID_x', 'DEST_x',
       'DEST_ID_x', 'CODESHARE_x', 'STOPS_x', 'EQUIPMENT_x', 'AIRLINE_y',
       'AIRLINE_ID_y', 'SOURCE_y', 'SOURCE_ID_y', 'DEST_y', 'DEST_ID_y',
       'CODESHARE_y', 'STOPS_y', 'EQUIPMENT_y', 'AIRLINE', 'AIRLINE_ID',
       'SOURCE', 'SOURCE_ID', 'DEST', 'DEST_ID', 'CODESHARE', 'STOPS',
       'EQUIPMENT'],
      dtype='object')

In [54]:
list(twotransfer_routes)

['AIRLINE_x',
 'AIRLINE_ID_x',
 'SOURCE_x',
 'SOURCE_ID_x',
 'DEST_x',
 'DEST_ID_x',
 'CODESHARE_x',
 'STOPS_x',
 'EQUIPMENT_x',
 'AIRLINE_y',
 'AIRLINE_ID_y',
 'SOURCE_y',
 'SOURCE_ID_y',
 'DEST_y',
 'DEST_ID_y',
 'CODESHARE_y',
 'STOPS_y',
 'EQUIPMENT_y',
 'AIRLINE',
 'AIRLINE_ID',
 'SOURCE',
 'SOURCE_ID',
 'DEST',
 'DEST_ID',
 'CODESHARE',
 'STOPS',
 'EQUIPMENT']